# Basic Preparation

### Logging

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Imports

In [2]:
from typing import Any, List, Mapping, Optional

from IPython.display import Markdown, display

In [3]:
import torch
from langchain.llms.base import LLM
from llama_index import LLMPredictor, PromptHelper, ServiceContext, SimpleDirectoryReader
from transformers import pipeline

### Define Custom LLM

Define Prompt Helper

In [ ]:
max_input_size = 1024   # Set maximum input size
num_output = 256        # Set number of output tokens
max_chunk_overlap = 16  # Set maximum chunk overlap

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [ ]:
class CustomLLM(LLM):
    model_name = 'sberbank-ai/ruT5-large'  # 1024
    # model_name = 'cointegrated/rut5-base-multitask'  # 768
    pipeline = pipeline(
        'text2text-generation',
        model=model_name,
        device='cuda:0',
        model_kwargs={'torch_dtype': torch.bfloat16},
    )

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]['generated_text']

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {'name_of_model': self.model_name}

    @property
    def _llm_type(self) -> str:
        return 'custom'

In [ ]:
llm_predictor = LLMPredictor(llm=CustomLLM())
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor, prompt_helper=prompt_helper
)

# Indexing & Querying

### GPT List Index

In [ ]:
from llama_index import GPTListIndex

In [ ]:
index_path = 'indices/partial/index_list.json'

In [ ]:
# Build linked list index
documents = SimpleDirectoryReader('data/partial').load_data()
index = GPTListIndex.from_documents(documents, service_context=service_context)

# Save index
index.save_to_disk(index_path)

In [ ]:
# Load index from disk
index = GPTListIndex.load_from_disk(index_path, service_context=service_context)

In [ ]:
# Query index
response = index.query(
    'В каком году Уголовный кодекс Российской Федерации был принят Государственной Думой?'
)
display(Markdown(f'<b>{response}</b>'))

### GPT Tree Index

In [ ]:
from llama_index import GPTTreeIndex

In [ ]:
index_path = 'indices/partial/index_tree.json'

In [ ]:
# Build tree index
documents = SimpleDirectoryReader('data/partial').load_data()
index = GPTTreeIndex.from_documents(documents, service_context=service_context)

# Save index
index.save_to_disk(index_path)

In [ ]:
# Load index from disk
index = GPTTreeIndex.load_from_disk(index_path, service_context=service_context)

#### Default Leaf Traversal

In [ ]:
# Query index
response = index.query(
    'В каком году Уголовный кодекс Российской Федерации был принят Государственной Думой?'
)
display(Markdown(f'<b>{response}</b>'))

#### Leaf Traversal with child_branch_factor=2

In [ ]:
# Query index
response = index.query(
    'В каком году Уголовный кодекс Российской Федерации был принят Государственной Думой?',
    child_branch_factor=2
)
display(Markdown(f'<b>{response}</b>'))

#### Tree Index with a custom Summary Prompt, directly retrieve answer from root node

In [ ]:
from llama_index import SummaryPrompt

In [ ]:
query_str = 'В каком году Уголовный кодекс Российской Федерации был принят Государственной Думой?'
SUMMARY_PROMPT_TMPL = (
    'Ниже представлена контекстная информация. \n'
    '---------------------\n'
    '{context_str}'
    '\n---------------------\n'
    'Исходя из контекстной информации и не имея предварительных знаний, '
    f'ответь на вопрос: {query_str}\n'
)
SUMMARY_PROMPT = SummaryPrompt(SUMMARY_PROMPT_TMPL)

In [ ]:
index_path = 'indices/partial/index_tree_query.json'

In [ ]:
# Build tree index
index_with_query = GPTTreeIndex.from_documents(
    documents, service_context=service_context, summary_template=SUMMARY_PROMPT
)

# Save index
index_with_query.save_to_disk(index_path)

In [ ]:
# Load index from disk
index_with_query = GPTTreeIndex.load_from_disk(index_path, service_context=service_context)

In [ ]:
# Query index
response = index.query(query_str, mode='retrieve')
display(Markdown(f'<b>{response}</b>'))

### GPT Keyword Table Index

In [26]:
from llama_index import GPTKeywordTableIndex

In [ ]:
index_path = 'indices/partial/index_keyword.json'

In [ ]:
# Build keyword index
documents = SimpleDirectoryReader('data/partial').load_data()
index = GPTKeywordTableIndex.from_documents(documents, service_context=service_context)

# Save index
index.save_to_disk(index_path)

In [ ]:
# Load index from disk
index = GPTListIndex.load_from_disk(index_path, service_context=service_context)

In [ ]:
# Query index
response = index.query(
    'В каком году Уголовный кодекс Российской Федерации был принят Государственной Думой?'
)
display(Markdown(f'<b>{response}</b>'))

### GPT Vector Index

In [ ]:
from llama_index import GPTSimpleVectorIndex

In [ ]:
index_path = 'indices/partial/index_vector.json'

In [ ]:
# Build vector index
documents = SimpleDirectoryReader('data/partial').load_data()
index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

# Save index
index.save_to_disk(index_path)

In [ ]:
# Load index from disk
index = GPTListIndex.load_from_disk(index_path, service_context=service_context)

In [ ]:
# Query index
response = index.query(
    'В каком году Уголовный кодекс Российской Федерации был принят Государственной Думой?'
)
display(Markdown(f'<b>{response}</b>'))